In [28]:
import numpy as np
import pandas as pd
import requests
from tqdm.notebook import tqdm
from libbgg.apiv1 import BGG 
## NOTE: substitute the following lines in apiv1.py file lines 68-69 because there is a bug in the code
"""
        return self.call('boardgame/{}'.format(
            ','.join([str(gid) for gid in game_ids])), d)
"""
import itertools

In [29]:
bbg = 'data/bgg.csv'

In [30]:
df = pd.read_csv(bbg)

In [31]:
df.head()

,game,title,rating
0,25984,Trivial Pursuit: Genus IV – Canadian Edition,3.0
1,25984,Trivial Pursuit: Genus IV – Canadian Edition,5.0
2,25984,Trivial Pursuit: Genus IV – Canadian Edition,3.0
3,25984,Trivial Pursuit: Genus IV – Canadian Edition,7.0
4,25984,Trivial Pursuit: Genus IV – Canadian Edition,6.0


In [32]:
id_list = df.game.unique()
len(id_list)

23931

In [33]:
def chunks(iterable, size):
    """Generate chunks of data"""
    it = iter(iterable)
    return iter(lambda: tuple(itertools.islice(it, size)), ())


In [34]:
chunk = list(chunks(id_list, 1000))

In [35]:
conn = BGG()

In [36]:
games = []

In [37]:
#Chunks are needed to avoid connection issues with BGG server

for i in chunk:
    results = conn.get_game(game_ids=i , stats=True)
    #create list of dictionaries with game information
    for rec in results['boardgames']['boardgame']:
        record = {}
        #game id
        record['game'] = rec['objectid']
        #game title
        if not isinstance(rec['name'], list):
            record['title'] = rec['name']['TEXT']
        else:
            for i in rec['name']:
                try:
                    i['primary']
                    record['title'] =i['TEXT']
                except:
                    pass
        #game statistics: rank rating and count
        if not isinstance(rec['statistics']['ratings']['ranks']['rank'], list):
            rate=rec['statistics']['ratings']['ranks']['rank']
        else:
            rate=[value for index, value in
                   enumerate(rec['statistics']['ratings']['ranks']['rank'])
                   if value['name'] == 'boardgame'][0]
        record['rank']= rate['value']
        record['rating'] = rate['bayesaverage']
        record['count'] = rec['statistics']['ratings']['usersrated']['TEXT']
        games.append(record)


In [38]:
games_df = pd.DataFrame(games)

In [40]:
games_df.to_csv('data/bgg_true_stats.csv',index=False)

In [41]:
games_df.head()

,game,title,rank,rating,count
0,25984,Trivial Pursuit: Genus IV – Canadian Edition,Not Ranked,Not Ranked,29
1,103339,Reluctant Enemies: Operation Exporter,3419,5.82593,259
2,132758,Fool's Gold,3388,5.82964,458
3,16497,Rat Hot,3498,5.81678,1019
4,195194,Horizons,3318,5.83852,544


In [42]:
games_df.describe()

,game,title,rank,rating,count
count,23923,23923,23923,23923,23923
unique,23923,23370,17009,13610,3019
top,255594,Grand Prix,Not Ranked,Not Ranked,10
freq,1,5,6915,6011,351
